<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9a0c631c7e7d0bc058e7c5b70ce7e765527b1e10c5d0d5f0d194df8b04aca546
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 13:20:44
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -82.85 K (-0.58%)
Current PnL: -24.28 L (-15.91%)
CY Booked + Current PnL: -10.39 L (-6.81%)
-------------------
Total profit:  1.48 L
Total loss:  -25.76 L
-------------------
Total Booked + Current PnL: 16.59 L (13.2%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.82%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.70 L (63.39%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.11,-9.24,10.20,0.02,16160.0,-16124.0,158435.0,141.63,65.0,M-SC,14.23,234.0,-1.00,1.12,37.81,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.12,-16.66,20.10,0.09,16882.0,-16786.0,83991.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.59,10.35,OX40N,NTT,DURABLES
51,MASFIN,398.61,0.28,-2.28,24.90,22.05,23841.0,-2235.0,95745.0,-15.54,59.0,H-SC,6.73,164.0,-0.09,0.68,39.17,XR,ATH,FINANCE
43,ITC,452.00,-0.30,-2.33,13.07,10.44,25549.0,-4660.0,195478.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.11,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.74,5.21,14.24,20.19,26117.0,9085.0,183405.0,-17.43,45.0,X-MC,6.72,68.0,0.35,1.30,21.11,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,7.69,-45.38,209.53,69.05,152276.0,-60391.0,72675.0,-55.87,58.0,H-SC,23.68,140.0,-0.40,0.51,12.08,SR,ATH,IT
13,BSOFT,831.70,3.83,-23.77,97.81,50.79,104867.0,-33438.0,107215.0,-2.20,71.0,H-SC,5.89,151.0,-0.32,0.76,24.83,XR,ATH,IT
5,ASIANPAINT,3460.25,3.03,3.26,17.12,20.95,44510.0,8219.0,259987.0,2.18,79.0,X-LC,16.56,31.0,0.18,1.84,40.06,X40,ATH,PAINTS
85,WIPRO,420.00,1.93,4.59,64.70,72.27,114669.0,7784.0,177232.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.05,XR,NTT,IT
74,TCS,4389.97,1.46,-13.56,37.99,19.28,113610.0,-46904.0,299052.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.11,10.55,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-5.26,-39.44,86.42,12.90,114360.0,-86166.0,132330.0,94.71,46.0,H-SC,12.18,161.0,-0.75,0.94,10.03,OX40N,NTT,FINANCE
24,EASEMYTRIP,26.40,-3.75,-54.64,221.17,45.70,177403.0,-96604.0,80211.0,-16.61,61.0,M-SC,23.42,205.0,-0.54,0.57,15.61,XY24,NTT,TRAVEL
22,DIXON,18931.72,-2.95,-9.67,34.59,21.58,43789.0,-13545.0,126594.0,-54.45,29.0,X-MC,5.87,56.0,-0.31,0.89,12.08,X40N,ATH,IT
28,GREENPANEL,537.00,-2.75,-35.85,123.14,43.14,144618.0,-65636.0,117442.0,162.92,28.0,M-SC,6.84,240.0,-0.45,0.83,10.47,XY24,NTT,MISC
18,COFFEEDAY,80.00,-2.75,-42.51,135.29,35.27,88317.0,-48269.0,65280.0,-55.73,27.0,L-SC,14.78,270.0,-0.55,0.46,59.77,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.26,0.22,106.28,106.72,131363.0,265.0,123601.0,-50.44,40.0,H-SC,1.73,139.0,0.00,0.87,23.64,AR,ATH,MISC
37,IEX,219.00,-0.23,0.58,47.77,48.63,96069.0,1153.0,201107.0,-33.16,65.0,H-SC,14.79,136.0,0.01,1.42,12.20,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.34,-5.98,121.13,107.91,168151.0,-8825.0,138819.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.11,-9.24,10.20,0.02,16160.0,-16124.0,158435.0,141.63,65.0,M-SC,14.23,234.0,-1.00,1.12,37.81,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.12,-16.66,20.10,0.09,16882.0,-16786.0,83991.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.59,10.35,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.13,-23.12,47.60,13.47,98682.0,-62352.0,207315.0,-68.31,31.0,H-SC,2.15,158.0,-0.63,1.47,5.41,XY24,NTT,PAINTS
67,SIS,528.00,0.88,-24.62,60.88,21.26,50951.0,-27341.0,83691.0,1973.58,46.0,H-SC,3.92,166.0,-0.54,0.59,13.07,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.93,4.59,64.70,72.27,114669.0,7784.0,177232.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.05,XR,NTT,IT
37,IEX,219.00,-0.23,0.58,47.77,48.63,96069.0,1153.0,201107.0,-33.16,65.0,H-SC,14.79,136.0,0.01,1.42,12.20,XR,NTT,MISC
38,INDIAMART,4810.62,0.26,0.22,106.28,106.72,131363.0,265.0,123601.0,-50.44,40.0,H-SC,1.73,139.0,0.00,0.87,23.64,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.93,4.59,64.70,72.27,114669.0,7784.0,177232.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.05,XR,NTT,IT
37,IEX,219.00,-0.23,0.58,47.77,48.63,96069.0,1153.0,201107.0,-33.16,65.0,H-SC,14.79,136.0,0.01,1.42,12.20,XR,NTT,MISC
38,INDIAMART,4810.62,0.26,0.22,106.28,106.72,131363.0,265.0,123601.0,-50.44,40.0,H-SC,1.73,139.0,0.00,0.87,23.64,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.71,-1.30,37.41,35.63,77385.0,-2718.0,206856.0,-15.41,42.0,H-MC,3.63,119.0,-0.04,1.46,15.16,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.34,-5.98,121.13,107.91,168151.0,-8825.0,138819.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.21,-20.48,86.69,48.45,138219.0,-41065.0,159441.0,-26.10,14.0,X-MC,8.84,64.0,-0.30,1.13,0.21,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.11,-37.70,117.03,35.20,93808.0,-48513.0,80157.0,5.13,18.0,X-SC,14.24,92.0,-0.52,0.57,0.11,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-1.58,-9.59,39.44,26.07,58387.0,-15700.0,148040.0,-33.39,27.0,X-MC,10.52,55.0,-0.27,1.05,0.00,X40,ATH,APPARELS
8,AWL,485.00,-0.60,-21.54,95.64,53.50,226422.0,-65007.0,236744.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.15,XY24,NTT,FMCG
23,DMART,5391.45,-1.63,-7.39,38.34,28.12,74711.0,-15542.0,194865.0,-20.12,28.0,X-LC,4.36,19.0,-0.21,1.38,14.49,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.70,-22.72,112.40,64.15,206704.0,-54051.0,183900.0,-55.00,46.0,X-SC,1.45,82.0,-0.26,1.30,3.16,XY24,BTT,CEMENT
43,ITC,452.0,-0.30,-2.33,13.07,10.44,25549.0,-4660.0,195478.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.11,X40,NTT,FMCG
15,CAMS,4762.0,0.50,1.42,23.05,24.79,58874.0,3564.0,255420.0,-3.25,48.0,X-SC,1.93,86.0,0.06,1.81,24.88,X40N,NTT,MISC
8,AWL,485.0,-0.60,-21.54,95.64,53.50,226422.0,-65007.0,236744.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.15,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.96,1.23,29.38,30.97,52563.0,2168.0,178906.0,-22.64,47.0,X-MC,2.17,75.0,0.04,1.26,13.94,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-2.44,-20.05,77.00,41.51,162124.0,-52815.0,210550.0,-4.76,31.0,X-MC,7.32,61.0,-0.33,1.49,0.00,XY25,ATH,FMCG
54,PAGEIND,51605.07,-1.58,-9.59,39.44,26.07,58387.0,-15700.0,148040.0,-33.39,27.0,X-MC,10.52,55.0,-0.27,1.05,0.00,X40,ATH,APPARELS
56,QUESS,424.00,-0.31,-29.68,103.12,42.84,47140.0,-19292.0,45714.0,-53.72,38.0,X-SC,37.40,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,-0.41,-5.53,41.58,33.76,78886.0,-11100.0,189720.0,-33.32,30.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG
11,BATAINDIA,2096.00,0.11,-37.70,117.03,35.20,93808.0,-48513.0,80157.0,5.13,18.0,X-SC,14.24,92.0,-0.52,0.57,0.11,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.31,-29.68,103.12,42.84,47140.0,-19292.0,45714.0,-53.72,38.0,X-SC,37.40,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.79,-47.80,191.78,52.31,145317.0,-69387.0,75773.0,-44.44,33.0,X-SC,5.04,91.0,-0.48,0.54,1.43,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.11,-37.70,117.03,35.20,93808.0,-48513.0,80157.0,5.13,18.0,X-SC,14.24,92.0,-0.52,0.57,0.11,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.95,-9.67,34.59,21.58,43789.0,-13545.0,126594.0,-54.45,29.0,X-MC,5.87,56.0,-0.31,0.89,12.08,X40N,ATH,IT
52,MEDANTA,1486.00,-1.63,0.58,23.36,24.08,29828.0,738.0,127688.0,-7.57,39.0,X-SC,5.83,89.0,0.02,0.90,20.31,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,1.46,-13.56,37.99,19.28,113610.0,-46904.0,299052.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.11,10.55,X40,ATH,IT
41,INFY,2275.00,0.84,7.16,44.53,54.87,149302.0,22395.0,335283.0,-16.73,63.0,X-LC,2.70,2.0,0.15,2.37,16.03,X40,BTT,IT
76,TMPV,600.00,-0.56,-14.28,65.86,42.18,156053.0,-39464.0,236946.0,-23.08,35.0,X-LC,2.74,3.0,-0.25,1.67,2.64,XY24,NTT,AUTO
82,VBL,671.64,-0.29,-3.04,39.92,35.67,122251.0,-9602.0,306240.0,-14.52,60.0,X-LC,5.38,4.0,-0.08,2.16,10.39,X40N,ATH,FMCG
43,ITC,452.00,-0.30,-2.33,13.07,10.44,25549.0,-4660.0,195478.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.11,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.09,-37.28,111.89,32.89,54179.0,-28784.0,48422.0,-697.80,60.0,L-MC,6.01,259.0,-0.53,0.34,33.35,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.89,-15.65,115.37,81.67,91293.0,-14679.0,79131.0,7111.11,42.0,L-SC,18.72,271.0,-0.16,0.56,53.24,XR,NTT,CERAMICS
51,MASFIN,398.61,0.28,-2.28,24.90,22.05,23841.0,-2235.0,95745.0,-15.54,59.0,H-SC,6.73,164.0,-0.09,0.68,39.17,XR,ATH,FINANCE
13,BSOFT,831.70,3.83,-23.77,97.81,50.79,104867.0,-33438.0,107215.0,-2.20,71.0,H-SC,5.89,151.0,-0.32,0.76,24.83,XR,ATH,IT
70,SURYODAY,216.00,-0.98,-19.75,53.31,23.03,76611.0,-35363.0,143708.0,55.70,48.0,H-SC,9.98,167.0,-0.46,1.02,42.59,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.11,-9.24,10.20,0.02,16160.0,-16124.0,158435.0,141.63,65.0,M-SC,14.23,234.0,-1.00,1.12,37.81,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,3.03,3.26,17.12,20.95,44510.0,8219.0,259987.0,2.18,79.0,X-LC,16.56,31.0,0.18,1.84,40.06,X40,ATH,PAINTS
13,BSOFT,831.70,3.83,-23.77,97.81,50.79,104867.0,-33438.0,107215.0,-2.20,71.0,H-SC,5.89,151.0,-0.32,0.76,24.83,XR,ATH,IT
50,LTIM,7201.88,-0.06,12.62,16.90,31.66,46851.0,31071.0,277222.0,8.32,69.0,H-LC,14.93,43.0,0.66,1.96,53.82,X200,ATH,IT
66,SIEMENS,4671.50,-0.62,-10.26,39.87,25.51,66583.0,-19095.0,167000.0,2.41,65.0,H-LC,2.72,49.0,-0.29,1.18,23.07,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.07
1,25,44.84
2,50,75.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.40
MC    28.88
LC    25.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.57
X40      21.34
X40N     11.92
XR        9.73
AR        9.07
XY25      9.06
OX40N     7.59
X200      1.96
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.95
X-MC    22.43
X-LC    20.35
M-SC    11.88
X-SC     8.19
H-MC     4.74
H-LC     3.14
L-SC     1.38
M-MC     1.37
M-LC     1.25
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.71,-6.43,40.85
IT,14.14,-12.92,71.21
FINANCE,9.32,-17.70,67.43
MISC,7.44,-23.25,73.74
ELECTRICAL,6.01,-10.26,50.65
PAINTS,6.01,-8.67,24.81
INSURANCE,4.38,-1.99,37.93
PHARMA,3.89,-1.48,33.96
AUTO,3.37,-20.30,71.88


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3134894.0,21
XR,1288322.0,13
AR,1285227.0,10
X40,1020926.0,14
X40N,761806.0,9
OX40N,719692.0,10
XY25,359090.0,6
SR,276698.0,2
MH,76009.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3591899.0,25
M-SC,1377912.0,15
X-MC,1308421.0,16
X-LC,882226.0,11
X-SC,762198.0,8
H-MC,407236.0,3
L-SC,266212.0,3
M-LC,114669.0,1
H-LC,113434.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251757.0      6
           AR         882657.0      5
M-SC       XY24       786487.0      6
H-SC       XR         771774.0      7
X-MC       X40        461847.0      7
           XY24       388550.0      3
X-LC       X40        377627.0      5
H-SC       OX40N      333004.0      4
M-SC       OX40N      300086.0      5
X-SC       X40N       297074.0      3
           XY24       283672.0      3
H-SC       SR         276698.0      2
X-MC       X40N       267770.0      4
X-LC       XY24       229930.0      2
H-MC       AR         212738.0      2
X-LC       X40N       196962.0      2
H-MC       XY24       194498.0      1
X-MC       XY25       190254.0      2
X-SC       X40        181452.0      2
L-SC       XR         179610.0      2
M-SC       XR         168090.0      2
           AR         123249.0      2
M-LC       XR         114669.0      1
L-SC       OX40N       86602.0      1
X-LC       XY25        77707.0      2
H-SC       MH          76009.0      1
H-LC       AR          66583.0      1
L-MC       XR          54179.0      1
M-MC       XY25        51954.0      1
H-LC       X200        46851.0      1
L-LC       XY25        39175.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
